# Class 11: Cybersecurity with python

## Learning outcomes

At the completion of this unit students should be able to:
1.   Understand the concepts of encoding and decoding a plain text message
2. Apply the `cryptography` module to implement based symmetric cryptography

## 11.1 Terminology

- **Plain Text**:  is the text which is readable, and which we want to keep as a secret.
- **Cipher Text**: is the message obtained after applying cryptography on plain text.
- **Encryption (or encoding)**: is the process of converting plain text to cipher text.
- **Decryption (or decodinig)**: is the process of converting cipher text to plain text.
- **Symmetric encryption**: is when a plain text can be encrypted and decrypted using the same key.
- **AES (Advanced Encryption Standard)**: is a block cipher standardized by NIST. AES is both fast, and cryptographically strong. It is a good default choice for encryption. For more information: https://en.wikipedia.org/wiki/Advanced_Encryption_Standard.
- **KDF (Key Derivation Function)**: A cryptographic hash function that derives one or more secret keys from a password. For more information: https://en.wikipedia.org/wiki/Key_derivation_function.
- **Fernet**: is an implementation of symmetric cryptography. For more information: https://cryptography.io/en/latest/fernet/.


Here we will focus on the Fernet class for symmetric cryptography.

## 11.2 Python string prefixes: row and byte

A python string literal can be prefixed by a letter that indicates the type of characters in that string. String prefixes are documented here: https://docs.python.org/3/reference/lexical_analysis.html#string-and-bytes-literals. Will briefly discuss it in today's class.

There are two types of string literals (or string values) in python: row string literals, which are the ones we have been dealing with so far, and byte string literals.

Bytes string literals are always prefixed with 'b' or 'B'. For example, this is a byte string literal: `b"Hi there!"`, where as this is a row string literatal: `"Hi there!"`.

A byte string literal is an instance of the *bytes* type instead of the *str* type. Let's check that using the `type()` function.

In [0]:
a = b"Hi there!"
print(type(a))

a = r"Hi there!"
print(type(a))

<class 'bytes'>
<class 'str'>


In [0]:
b = b'Some text string'
print(b)
print('Decode byte to str: ',b.decode('utf-8'))

r = r'Some text string'
print(r)
print('Encode str to byte', r.encode('utf-8'))



b'Some text string'
Some text string
Some text string
b'Some text string'


They may only contain ASCII characters; bytes with a numeric value of 128 or greater must be expressed with escapes.

### Generation of random numbers and bytes

Random number and string generation is very impotant in cryptography because it enables key generators to have unexpected (random) features which cannot be prediced by adversaries. To generate a random number in python, we use the `random()` function.

In [0]:
import random
random.random()


# for i in range(10):
#   print(random.random())

0.8783550629952783

These numbers are random numbers in the range from 0 to 1. We can also generate a random byte string using the `urandom()` function in the `os` module. Note, in the example below, that `urandom()` receive an integer parameter which is the size of the random byte string.


In [0]:
import os
a = os.urandom(1)
print(a)

a = os.urandom(10)
print(a)

a = os.urandom(20)
print(a)

b'\xc7'
b'\x95\xf2_IK\xb4oe=\x8a'
b'\xc2\x99Dt\xc7\x02&OX\x1a5\xe4\xfb\xe6\xdf\xcc}KY\xfd'



## 11.3 Fernet

The most useful high-level secure primitive in cryptography is the Fernet implementation. Fernet is a standard for encrypting buffers in a way that follows best-practices cryptography. It is not suitable for very big files.

Fernet supports symmetric, or secret key, cryptography: the same key is used for encryption and decryption, and therefore must be kept safe.



In [3]:
!pip3 install cryptography
import cryptography

from cryptography.fernet import Fernet

key = Fernet.generate_key()
fkey = Fernet(key)
print('The key object: ', key)
cipher_text = fkey.encrypt(b"I am working from home.")
print("Cipher: ",cipher_text)
plain_text = fkey.decrypt(cipher_text)
print("Plain: ",plain_text)

#b'gAAAAABetMnu3xJb0aLsMkGPcIP4zseosJLkYqOOjHi7guLzuc7sRwrScSdqd3TcB7cs5Ah0rJeJVCzrRQorl



The key object:  b'4QI0afgIQeZwEubJwd60Wa42ObrLvBlsSsKFPaKWyI8='
Cipher:  b'gAAAAABevdUk4bW_JxFpbflWHyhbgSZp3wyHU4LJh9fi3u1hJj_8nbi_zeFXPqoKXMjjuihOElbjLCAZU7EiaXkBt3-xbnv1yjGr6g96s_umUrzQfzK4iww='
Plain:  b'I am working from home.'


You will get slightly different values if you encrypt on your machine. Not only because (I hope) you generated a different key from me, but because Fernet concatenates the value to be encrypted with some randomly generated buffer. This is one of the "best practices" I alluded to earlier: it will prevent an adversary from being able to tell which encrypted values are identical, which is sometimes an important part of an attack.

Note that this only encrypts and decrypts byte strings. In order to encrypt and decrypt text strings, they will need to be encoded and decoded, usually with UTF-8.


In [0]:
st = "I am working from home."
st=st.encode('utf-8')

from cryptography.fernet import Fernet
key = Fernet.generate_key()
fkey = Fernet(key)
print('The key object: ', fkey)
cipher_text = fkey.encrypt(st)
print("Cipher: ",cipher_text)
plain_text = fkey.decrypt(cipher_text)
print("Plain: ",plain_text)

The key object:  <cryptography.fernet.Fernet object at 0x7f41c9356390>
Cipher:  b'gAAAAABetDPu8FXJ4Fihi1VLfoItolCmjofpnUExZ6un0xaVkqoT8a-kgdtDbLJnM3bl_rRiZ_-cvd0Qvw9PyKN8KrVNbrnymsJwBllg-7ooZPbs26l0sdo='
Plain:  b'I am working from home.'


## 11.3 Fernet key from a password

You can create a new key based on a password. There are many differernt `cryptography` classes that can *derive* a key based on a password. Here will use one of them, `PBKDF2HMAC`.

In [0]:
import base64
from cryptography.fernet import Fernet
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
password = b"my complicated password"

kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=os.urandom(10),
    iterations=100000,
    backend=default_backend()
)
key = base64.urlsafe_b64encode(kdf.derive(password))
fkey = Fernet(key)
print('The key object: ', fkey)
cipher_text = fkey.encrypt(st)
print("Cipher: ",cipher_text)
plain_text = fkey.decrypt(cipher_text)
print("Plain: ",plain_text)

The key object:  <cryptography.fernet.Fernet object at 0x7f41c936aa58>
Cipher:  b'gAAAAABetDuEgar20CH6Kkk1BFetMUvSflqoP1d7eUKaVd1L2CxT8lZRjqe5QGhWEO2ryrOTYj7U0Xm9CBmNLfFQHvP4HK8GzIF-obgJURRmvQCW8HBVA4w='
Plain:  b'I am working from home.'
